In [ ]:
import numpy as np
import gillespy2
from gillespy2 import Model, Species, Parameter, Reaction, Event, \
                      EventTrigger, EventAssignment, RateRule, \
                      AssignmentRule, FunctionDefinition
from gillespy2 import SSACSolver
# from gillespy2 import TauLeapingCSolver
# from gillespy2 import ODECSolver
# from gillespy2 import TauHybridSolver

# Toggle_Switch

In [ ]:
class ToggleSwitch(Model):
    def __init__(self, parameter_values=None):
        Model.__init__(self, name="Toggle_Switch")
        self.volume = 1

        # Parameters
        self.add_parameter(Parameter(name="alpha1", expression="10"))
        self.add_parameter(Parameter(name="alpha2", expression="10"))
        self.add_parameter(Parameter(name="beta", expression="2"))
        self.add_parameter(Parameter(name="gamma", expression="2"))
        self.add_parameter(Parameter(name="mu", expression="1"))

        # Variables
        self.add_species(Species(name="A", initial_value=2, mode="discrete"))
        self.add_species(Species(name="B", initial_value=2, mode="discrete"))

        # Reactions
        self.add_reaction(Reaction(name="cu", reactants={}, products={'A': 1}, propensity_function="alpha1/(1+pow(B, beta))"))
        self.add_reaction(Reaction(name="cv", reactants={}, products={'B': 1}, propensity_function="alpha2/(1+pow(A, gamma))"))
        self.add_reaction(Reaction(name="du", reactants={'A': 1}, products={}, rate=self.listOfParameters["mu"]))
        self.add_reaction(Reaction(name="dv", reactants={'B': 1}, products={}, rate=self.listOfParameters["mu"]))

        # Timespan
        self.timespan(np.arange(0, 250, 1))

In [ ]:
model = ToggleSwitch()

# Simulation Parameters

In [ ]:
def configure_simulation():
    solver = SSACSolver(model=model)
    kwargs = {
        "solver":solver,
        "number_of_trajectories":100,
        # "seed":None,
        # "tau_tol":0.03,
        # "integrator_options":{'rtol': 0.001, 'atol': 1e-06},
    }
    return kwargs

# Model Inference

In [ ]:
%load_ext autoreload
%autoreload 2

from tsfresh.feature_extraction.settings import MinimalFCParameters
from sciope.utilities.priors import uniform_prior
from sciope.utilities.summarystats import auto_tsfresh
from sciope.utilities.distancefunctions import naive_squared
from sciope.inference.abc_inference import ABC
from sklearn.metrics import mean_absolute_error
from dask.distributed import Client

## Generate some fixed(observed) data based on default parameters of the model

In [ ]:
kwargs = configure_simulation()
fixed_data = model.run(**kwargs)

In [ ]:
# Reshape the data and remove timepoints array
fixed_data = fixed_data.to_array()
fixed_data = np.asarray([x.T for x in fixed_data])
fixed_data = fixed_data[:, 1:, :]

## Define prior distribution

In [ ]:
# take default from mode 1 as reference
default_param = np.array(list(model.listOfParameters.items()))[:, 1]

bound = []
for exp in default_param:
    bound.append(float(exp.expression))

# Set the bounds
bound = np.array(bound)
dmin = bound * 0.1
dmax = bound * 2.0

# Here we use uniform prior
uni_prior = uniform_prior.UniformPrior(dmin, dmax)

## Define simulator

In [ ]:
def get_variables(params, model):
    # params - array, need to have the same order as model.listOfParameters
    variables = {}
    for e, pname in enumerate(model.listOfParameters.keys()):
        variables[pname] = params[e]
    return variables

# Here we use the GillesPy2 Solver
def simulator(params, model):
    variables = get_variables(params, model)

    res = model.run(**kwargs, variables=variables)
    res = res.to_array()
    tot_res = np.asarray([x.T for x in res]) # reshape to (N, S, T)
    # should not contain timepoints
    tot_res = tot_res[:, 1:, :]

    return tot_res

# Wrapper, simulator function to abc should should only take one argument (the parameter point)
def simulator2(x):
    return simulator(x, model=model)

## Define summary statistics and distance function

In [ ]:
# Function to generate summary statistics
summ_func = auto_tsfresh.SummariesTSFRESH()

# Distance
ns = naive_squared.NaiveSquaredDistance()

## Start local cluster using dask client

In [ ]:
c = Client()

## Start abc instance

In [ ]:
abc = ABC(fixed_data, sim=simulator2, prior_function=uni_prior, summaries_function=summ_func.compute, distance_function=ns)

In [ ]:
# First compute the fixed(observed) mean
abc.compute_fixed_mean(chunk_size=2)

In [ ]:
res = abc.infer(num_samples=100, batch_size=10, chunk_size=2)

In [ ]:
mae_inference = mean_absolute_error(bound, abc.results['inferred_parameters'])